In [1]:
# !pip install lastmile-eval --upgrade

In [2]:
# Setup Cell
from lastmile_eval.rag.debugger.tracing import (
    get_lastmile_tracer,
)
from lastmile_eval.rag.debugger.api import (
    LastMileTracer)

# Replace token here
lastmile_api_token = "enter token here!"

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="my-tracer5", # project name
    lastmile_api_token=lastmile_api_token,
    initial_params={"global_param": "This param key appear in all traces"},
)

/Users/ankush/anaconda3/envs/main/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-13 01:08:29,873 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 01:08:29,976 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=my-tracer5 HTTP/1.1" 200 41
2024-05-13 01:08:29,978 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 01:08:30,059 - https://lastmileai.dev:443 "POST /api/evaluation_projects/create HTTP/1.1" 200 247


In [3]:
# Logs feedback to the Project
tracer.log_feedback("Rag debugger retrieves context nicely")

2024-05-13 01:08:30,069 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 01:08:30,147 - https://lastmileai.dev:443 "POST /api/evaluation_feedback/create HTTP/1.1" 200 355


{'id': 'clw4i73uu0034pbesxeb58l1x',
 'createdAt': '2024-05-13T05:08:30.150Z',
 'updatedAt': '2024-05-13T05:08:30.150Z',
 'creatorId': 'clkrgxm850004phi6ee5mvhd1',
 'feedback': {'text': 'Rag debugger retrieves context nicely'},
 'projectId': 'clw4i73sb0026qynx1jxe1lvz',
 'metadata': None,
 'traceId': None,
 'spanId': None,
 'organizationId': None,
 'visibility': 'MEMBER',
 'active': True}

Logging Online Feedback Post Span and Trace Create

In [ ]:
from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.tracing.sdk import get_span_id, get_trace_id
import os
from opentelemetry import trace
import opentelemetry
from openai import OpenAI
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="generate_riddle", # project name
)

@tracer.start_as_current_span("generate_riddle")
def generate()-> str:
    """
    Generates a riddle using OpenAI's GPT-3.5-turbo model.
    If the OPENAI_API_KEY environment variable is not set, a default riddle is returned.
    The generated riddle is returned as a JSON string.
    """
    riddle = ""
    if os.environ.get("OPENAI_API_KEY") is not None:
        response = OpenAI().chat.completions.create(messages = [{"role": "user", "content":"tell me a riddle"}], model = "gpt-3.5-turbo")
        riddle = response.choices[0].message.content or "debugging me can be quite a trick."
    else:
        riddle = "I'm a key that unlocks no doors, But fill me in, and power soars. In the realm where AI plays"

    # Store key-value pair in the LastMileTracer, where the key is the riddle and the value is the spanID
    # This will allow us to log online feedback for the generated riddle
    try:
        # Store the span id and the trace ID with the generated riddle
        tracer.store_key_value_pair(key=riddle, span_id=get_span_id(), trace_id= get_trace_id()) 
    except Exception as e:
        print(e)
    return riddle
riddle = generate()

# We stored the spanID in the previous step, so we can retrieve it using the riddle as the key
trace_id, span_id = tracer.read_key_value_pair(key=riddle)
print("read", trace_id, span_id)

2024-05-13 00:41:02,865 - Overriding of current TracerProvider is not allowed


2024-05-13 00:41:02,867 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 00:41:02,952 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=generate_riddle HTTP/1.1" 200 350
2024-05-13 00:41:02,954 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 00:41:03,031 - https://lastmileai.dev:443 "POST /api/evaluation_key_value_store/create HTTP/1.1" 200 278
2024-05-13 00:41:03,066 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-13 00:41:03,067 - Starting new HTTPS connection (1): lastmileai.dev:443


generateL  427348558040722529 335335113912571933472500984089549150308


2024-05-13 00:41:03,159 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 495
2024-05-13 00:41:03,162 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 00:41:03,255 - https://lastmileai.dev:443 "GET /api/evaluation_key_value_store/read?projectId=clvzt661o007dpbt9n0nq3o1z&key=test9 HTTP/1.1" 200 278


read 335335113912571933472500984089549150308 427348558040722529


In [ ]:
riddle = generate()

# We stored the spanID in the previous step, so we can retrieve it using the riddle as the key
trace_id, span_id = tracer.read_key_value_pair(key=riddle)

tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="The riddle was too easy.")  

2024-05-13 00:41:17,335 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-13 00:41:17,517 - https://lastmileai.dev:443 "POST /api/evaluation_feedback/create HTTP/1.1" 200 385


{'id': 'clw4h8443001fqp5bci3a744f',
 'createdAt': '2024-05-13T04:41:17.524Z',
 'updatedAt': '2024-05-13T04:41:17.524Z',
 'creatorId': 'clkrgxm850004phi6ee5mvhd1',
 'feedback': {'text': 'The riddle was too easy.'},
 'projectId': 'clvzt661o007dpbt9n0nq3o1z',
 'metadata': None,
 'traceId': 'fc4731983724909500ef432de0331864',
 'spanId': '5ee3f3fd54aec61',
 'organizationId': None,
 'visibility': 'MEMBER',
 'active': True}